In [1]:
import cohere
import json
import os

co = cohere.ClientV2(os.getenv("COHERE_API_KEY"))

In [2]:
from tavily import TavilyClient

tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))


def search_internet(query: str) -> dict:
    response = tavily_client.search(query)
    documents = []
    for document in response["results"]:
        documents.append(
            {"data": {"title": document["title"], "snippet": document["content"]}}
        )
    return documents


functions_map = {
    "search_internet": search_internet,
}

In [3]:
search_internet_tool = {
        "type": "function",
        "function": {
            "name": "search_internet",
            "description": "Searches the internet. Use this tool for general queries that would not be found in the developer documentation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query."
                    }
                },
                "required": ["query"]
            }
        }
}

tools = [search_internet_tool]

In [4]:
import datetime
date_today = datetime.datetime.now().strftime("%Y-%m-%d")

system_message=f"""You are a web search agent that helps users find information on the internet. You are equipped a tool to search the web for information. Use your search capabilities to provide accurate and up-to-date information to answer user queries.

Today's date is {date_today}
"""

In [10]:
model = "command-a-reasoning-08-2025"

def run_agent(query, messages=None):
    if messages is None:
        messages = []
        
    if "system" not in {m.get("role") for m in messages}:
        messages.append({"role": "system", "content": system_message})
    
    # Step 1: get user message
    print(f"QUESTION:\n{query}")
    print("="*50)
    
    messages.append({"role": "user", "content": query})
    
    step = 0
    max_steps = 10
    while step < max_steps:
        
        response = co.chat(model=model, messages=messages, tools=tools, temperature=0.1)
        
        if response.message.tool_calls:
            print("REASONING:")
            print(response.message.content[0].thinking, "\n")
            print("TOOL CALLS:")
            for tc in response.message.tool_calls:
                print(
                    f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}"
                )
            print("=" * 50)

            messages.append(response.message)

            # Step 3: Perform tool calls and get back tool results
            for tc in response.message.tool_calls:
                tool_result = functions_map[tc.function.name](
                    **json.loads(tc.function.arguments)
                )
                tool_content = []
                for data in tool_result:
                    tool_content.append(
                        {"type": "document", "document": {"data": json.dumps(data)}}
                    )
                messages.append(
                    {
                        "role": "tool", 
                        "tool_call_id": tc.id, 
                        "content": tool_content
                     }
                )
            step += 1
        else:
            messages.append(
                {
                    "role": "assistant", 
                    "content": response.message.content[1].text
                }
            )
            print("RESPONSE:")
            print(response.message.content[1].text)

            # Print citations (if any)
            verbose_source = False  # Change to True to display the contents of a source
            if response.message.citations:
                print("\nCITATIONS:")
                for citation in response.message.citations:
                    print(
                        f"Start: {citation.start}| End:{citation.end}| Text:'{citation.text}' "
                    )
                    print("Sources:")
                    for idx, source in enumerate(citation.sources):
                        print(f"{idx+1}. {source.id}")
                        if verbose_source:
                            print(f"{source.tool_output}")
                    print("\n")

            return messages

    return "Max steps reached"

In [11]:
query = "Who is the mayor of the capital of Ontario?"
query = "Who are the CEOs of the top 3 companies in the US by market cap?"

In [12]:
messages = run_agent(query)

QUESTION:
Who are the CEOs of the top 3 companies in the US by market cap?
REASONING:
Okay, the user is asking for the CEOs of the top 3 companies in the US by market capitalization. Let me think about how to approach this.

First, I need to identify the top 3 companies in the US based on market cap. Since market cap can change frequently, I should look for the most recent data. The best way to get this information is by doing an internet search. 

I'll start by using the search_internet tool with a query like "top 3 companies in the US by market cap 2025" to get the latest rankings. Once I have the company names, I can then search for each CEO individually. However, if the initial search already includes the CEO names, that would save some steps. Let me first perform the initial search to find the top companies. 

TOOL CALLS:
Tool name: search_internet | Parameters: {"query":"top 3 companies in the US by market cap"}
REASONING:
The search results for the top 3 US companies by market c

In [14]:
for m in messages:
    print(m, "\n")

{'role': 'system', 'content': "You are a web search agent that helps users find information on the internet. You are equipped a tool to search the web for information. Use your search capabilities to provide accurate and up-to-date information to answer user queries.\n\nToday's date is 2025-09-12\n"} 

{'role': 'user', 'content': 'Who are the CEOs of the top 3 companies in the US by market cap?'} 

role='assistant' tool_calls=[ToolCallV2(id='search_internet_61sxpknz6px9', type='function', function=ToolCallV2Function(name='search_internet', arguments='{"query":"top 3 companies in the US by market cap"}'))] tool_plan=None content=[ThinkingAssistantMessageResponseContentItem(value=None, type='thinking', thinking='Okay, the user is asking for the CEOs of the top 3 companies in the US by market capitalization. Let me think about how to approach this.\n\nFirst, I need to identify the top 3 companies in the US based on market cap. Since market cap can change frequently, I should look for the 